In [141]:
import re
import pandas as pd
import math
import numpy as np

In [142]:
#Mounting Google Drive:
#After running this cell a popup window will appear and requesting to select your  Google account and give the access permission.
#You can either use your personal Google account or your UIC Google account.
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [143]:
#After successfull mount, you can browse your Google Drive using linux commands:
!ls /content/gdrive/

MyDrive


In [144]:
#I have placed the data under mlcourse/hw1/agaricus in my Google Drive, so the address is:
#Yours would be different!

!head /content/gdrive/MyDrive/agaricus/agaricuslepiotatest1.csv

cap-shape-bell,cap-shape-conical,cap-shape-convex,cap-shape-flat,cap-shape-knobbed,cap-shape-sunken,cap-surface-fibrous,cap-surface-grooves,cap-surface-scaly,cap-surface-smooth,cap-color-brown,cap-color-buff,cap-color-cinnamon,cap-color-gray,cap-color-green,cap-color-pink,cap-color-purple,cap-color-red,cap-color-white,cap-color-yellow,bruises?-bruises,odor-almond,odor-anise,odor-creosote,odor-fishy,odor-foul,odor-musty,odor-none,odor-pungent,odor-spicy,gill-attachment-attached,gill-attachment-descending,gill-attachment-free,gill-attachment-notched,gill-spacing-close,gill-spacing-crowded,gill-spacing-distant,gill-size-broad,gill-color-black,gill-color-brown,gill-color-buff,gill-color-chocolate,gill-color-gray,gill-color-green,gill-color-orange,gill-color-pink,gill-color-purple,gill-color-red,gill-color-white,gill-color-yellow,stalk-shape-enlarging,stalk-root-bulbous,stalk-root-club,stalk-root-cup,stalk-root-equal,stalk-root-rhizomorphs,stalk-root-rooted,stalk-root-missing,stalk-surface-

In [145]:
#For this programs we are going to evaluate the ID3 on three different datasets:
#Change the path based on your directory address
path="/content/gdrive/MyDrive/"


#for agaricus we don't have a separate validation set, so we are going to reuse the training set.
agaricus = ["agaricus/agaricuslepiotatrain1.csv",
              "agaricus/agaricuslepiotatrain1.csv",
              "agaricus/agaricuslepiotatest1.csv"]

dataset1 = ["data_sets1/training_set.csv",
            "data_sets1/validation_set.csv",
            "data_sets1/test_set.csv"]



In [146]:
# Load data from a file. It returns a list of data points as well as the list of variable names
def read_data(filename):
    f = open(filename, 'r')
    p = re.compile(',')
    data = []
    header = f.readline().strip()
    varnames = p.split(header)
    namehash = {}
    for l in f:
        data.append([int(x) for x in p.split(l.strip())])
    return (data, varnames)

In [147]:
dataset = agaricus
train_data, varnames = read_data(path+dataset[0])
#the last element in the list is the class value."

In [148]:
#You can transfer the data to Pandas dataframe or directly load it with pd.read_csv but manupulating python lists would be easier.
#You can also use pandas to explore at the data.
data_df = pd.DataFrame(train_data, columns=varnames)
data_df

,cap-shape-bell,cap-shape-conical,cap-shape-convex,cap-shape-flat,cap-shape-knobbed,cap-shape-sunken,cap-surface-fibrous,cap-surface-grooves,cap-surface-scaly,cap-surface-smooth,...,population-several,population-solitary,habitat-grasses,habitat-leaves,habitat-meadows,habitat-paths,habitat-urban,habitat-waste,habitat-woods,poisonous
0,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
1,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
4,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0,0,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,1
5996,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
5997,0,0,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
5998,0,0,0,1,0,0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,1


In [ ]:
#We check the dimensions of our dataframe
data_df.shape

(6000, 123)

In [ ]:
#The first number in the shape is the xnumber of rows and the second one is the number of columns
#so we can quickly find the number examples:
data_df.shape[0]

6000

In [ ]:
#The class label is often the last column
classlabel = data_df.columns[-1]
print(classlabel)

poisonous


In [ ]:
#e.g. number of instances with positive class
data_df['poisonous'].sum()

2320

In [ ]:
#First attribute:
attr1 = data_df.columns[0]
print(attr1)

cap-shape-bell


In [ ]:
#Finding the rows that cap-shape-bell is one:
data_df[data_df['cap-shape-bell'] == 1]

,cap-shape-bell,cap-shape-conical,cap-shape-convex,cap-shape-flat,cap-shape-knobbed,cap-shape-sunken,cap-surface-fibrous,cap-surface-grooves,cap-surface-scaly,cap-surface-smooth,...,population-several,population-solitary,habitat-grasses,habitat-leaves,habitat-meadows,habitat-paths,habitat-urban,habitat-waste,habitat-woods,poisonous
2,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
7,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
12,1,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5935,1,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,1
5940,1,0,0,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,1
5947,1,0,0,0,0,0,0,0,0,1,...,1,0,1,0,0,0,0,0,0,1
5955,1,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,1


In [ ]:
#Counting the number of rows that cap-shape-bell is one
(data_df['cap-shape-bell'] == 1).sum()

302

In [ ]:
#Rows that have both  cap-shape-bell==1 and are population-several
data_df[(data_df['cap-shape-bell'] == 1) & (data_df['population-several'] == 1)]

,cap-shape-bell,cap-shape-conical,cap-shape-convex,cap-shape-flat,cap-shape-knobbed,cap-shape-sunken,cap-surface-fibrous,cap-surface-grooves,cap-surface-scaly,cap-surface-smooth,...,population-several,population-solitary,habitat-grasses,habitat-leaves,habitat-meadows,habitat-paths,habitat-urban,habitat-waste,habitat-woods,poisonous
4106,1,0,0,0,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,1
4826,1,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,1
4840,1,0,0,0,0,0,0,0,0,1,...,1,0,1,0,0,0,0,0,0,1
4843,1,0,0,0,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,1
4882,1,0,0,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,1
5002,1,0,0,0,0,0,0,0,0,1,...,1,0,1,0,0,0,0,0,0,1
5097,1,0,0,0,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,1
5131,1,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,1
5177,1,0,0,0,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,1
5224,1,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,1


In [ ]:
#Number of rows:
((data_df['cap-shape-bell'] == 1) & (data_df['population-several'] == 1)).sum()


44

In [ ]:
#Lets drop cap-shape-bell column:
new_data_df = data_df.drop(columns='cap-shape-bell');
new_data_df.columns[0] #the first attribute is different now
#also note that dataframe operations are immutable and you have to assign the output to a variable.


'cap-shape-conical'

In [ ]:
#Ok at this point I guess you know enough pandas to write the actual code.
#the main part that you gonna use dataframe is inside the build_tree function to split the data for each branch and
#in select_attr to do the required counting


In [149]:
#Here we have the neccessary data structure and auxilary functions for construcing the decision trees
#You don't need to modify the codes in this cell
class Node:
    """ Node class for a decision tree. """
    #var is the variable name for the node
    def __init__(self, var):
        self.var = var

    def classify(x):
        """ Handled by the subclasses. """
        return None

    def dump(self, indent):
        """ Handled by the subclasses. """
        return None


class Leaf(Node):
    #value is the label of the leaf node
    def __init__(self, value):
        Node.__init__(self, None);
        self.value = value

    def classify(self, x):
        return self.value

    def dump(self, indent):
        print(' %d' % self.value)


class Split(Node):
    #var: the variable that we create the split on
    #left and right are the branches for each side which are Nodes
    def __init__(self, var, left, right):
        Node.__init__(self, var)
        self.left = left
        self.right = right

    def classify(self, x):
        if x[self.var] == 0:
            return self.left.classify(x)
        else:
            return self.right.classify(x)

    #use to print out the tree recursively
    def dump(self, indent):
        if indent > 0:
            print('')
        for i in range(0, indent):
            print('| ', end='')
        print('%s = 0 :' % self.var,end='')
        self.left.dump(indent+1)
        for i in range(0, indent):
            print('| ', end='')
        print('%s = 1 :' % self.var,end='')
        self.right.dump(indent+1)


[link text]
Helper function computes entropy of Bernoulli distribution with parameter p

In [150]:
def entropy(p):
  if p == 0 or p == 1:
      return 0
  return -p * math.log2(p) - (1 - p) * math.log2(1 - p)


Compute information gain for a particular split, given the counts

ny_nxi : number of occurences of y=1 with x_i=1 for all i=1 to n (#y=1 $\wedge$ #xi =1)
(n is number of variables and each variable is binary)

nxi : number of occurrences of x_i=1 (#x_i)

ny : number of ocurrences of y=1

total: total number instances in this branch


In [151]:
def infogain(ny_nxi, nxi, ny, total):

    if nxi == 0:
      entropy_xi1 = 0
    else:
      entropy_xi1 = entropy(ny_nxi/nxi)

    if total - nxi == 0:
      entropy_xi0 = 0
    else:
      entropy_xi0 = entropy((ny-ny_nxi)/(total-nxi))

    info = (nxi / total) * entropy_xi1 + (1- (nxi / total)) * entropy_xi0

    return entropy(ny/total) - info

In [152]:
#split_df: the portion of the dataframe that routed to the current branch
def select_attr(split_df):

    target_col = split_df.columns[-1] # Last column

    total = len(split_df)
    ny = split_df[target_col].sum()


    selected_attr = None
    max_gain = -1


    for col in split_df.columns[:-1]:

        nxi = (split_df[col] == 1).sum() # Calculate the number of instances where the attribute is 1


        ny_nxi = ((split_df[col] == 1) & (split_df[target_col] == 1)).sum()  # Calculate the number of instances where both attribute and target are 1


        gain = infogain(ny_nxi, nxi, ny, total)

        # Update the selected attribute if the gain is greater than the maximum gain
        if gain > max_gain:
            selected_attr = col
            max_gain = gain

    return selected_attr, max_gain

Build tree in a top-down manner, selecting splits until we hit a pure leaf or all splits look bad.


In [153]:
#Recursive function for building the tree. Note that the vanilla ID3 stops when the nodes are pure (base condition for the recursion);
#config is potential dictionary of hyperparams that you can tune over validation dataset
#For potential list of hyperparams check here:
#https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier

def build_tree(data):
    total_instances = data.shape[0]
    positive_instances = (data[data.columns[-1]] == 1).sum()


    if positive_instances == 0:
        return Leaf(0)
    elif data.shape[1] == 1 or total_instances == positive_instances:
        return Leaf(1 if positive_instances >= total_instances - positive_instances else 0)
    else:

        attr, gain = select_attr(data) # Select the attribute with the highest information gain

        # Split the dataset into left and right subsets based on the selected attribute
        l_df = data[data[attr] == 0]
        r_df = data[data[attr] == 1]

        # Recursively build the left and right subtrees
        left_subtree = build_tree(l_df)
        right_subtree = build_tree(r_df)

        # Create a Split node representing the current node
        return Split(attr, left_subtree, right_subtree)



Build the decision tree

In [154]:
model = build_tree(data_df)

In [155]:
model.dump(0)

odor-foul = 0 :
| gill-size-broad = 0 :
| | odor-none = 0 :
| | | gill-spacing-close = 0 :
| | | | bruises?-bruises = 0 : 1
| | | | bruises?-bruises = 1 : 0
| | | gill-spacing-close = 1 : 1
| | odor-none = 1 :
| | | stalk-surface-above-ring-silky = 0 :
| | | | bruises?-bruises = 0 : 0
| | | | bruises?-bruises = 1 : 1
| | | stalk-surface-above-ring-silky = 1 : 1
| gill-size-broad = 1 :
| | spore-print-color-green = 0 : 0
| | spore-print-color-green = 1 : 1
odor-foul = 1 : 1


Calcuating the accuracy

In [156]:
#Assume the last column in the class label and returns one dataframe for labels and one for attributes
def separate_attributes_label(data):
  class_column = data.columns[-1]
  xdata = data.drop(columns=class_column);
  ydata = data[class_column];
  return xdata, ydata

In [157]:
xtrain, ytrain = separate_attributes_label(data_df);

In [158]:
def accuracy(model, xdata, ydata):
  correct = 0.0;
  for i in range(xdata.shape[0]):
    if model.classify(xdata.loc[i]) == ydata.loc[i]:
      correct += 1;
  return correct / xdata.shape[0]


In [159]:
print("Train Accuracy: {}".format(accuracy(model, xtrain, ytrain)))

Train Accuracy: 1.0


Now that we have all pieces working we can automate the training for any given dataset

In [160]:
def evaluate(dataset, config={}):
  train = pd.read_csv(path+dataset[0])
  val = pd.read_csv(path+dataset[1])
  test = pd.read_csv(path+dataset[2])

  xtrain, ytrain = separate_attributes_label(train);
  xval, yval = separate_attributes_label(val);
  xtest, ytest = separate_attributes_label(test);

  model = build_tree(train)
  model.dump(0)

  print("Train Accuracy: {}".format(accuracy(model, xtrain, ytrain)))
  print("Val Accuracy: {}".format(accuracy(model, xval, yval)))
  print("Test Accuracy: {}".format(accuracy(model, xtest, ytest)))

In [161]:
evaluate(agaricus)

odor-foul = 0 :
| gill-size-broad = 0 :
| | odor-none = 0 :
| | | gill-spacing-close = 0 :
| | | | bruises?-bruises = 0 : 1
| | | | bruises?-bruises = 1 : 0
| | | gill-spacing-close = 1 : 1
| | odor-none = 1 :
| | | stalk-surface-above-ring-silky = 0 :
| | | | bruises?-bruises = 0 : 0
| | | | bruises?-bruises = 1 : 1
| | | stalk-surface-above-ring-silky = 1 : 1
| gill-size-broad = 1 :
| | spore-print-color-green = 0 : 0
| | spore-print-color-green = 1 : 1
odor-foul = 1 : 1
Train Accuracy: 1.0
Val Accuracy: 1.0
Test Accuracy: 0.9792941176470589


In [ ]:
config={'min_example': 2}
evaluate(dataset1, config)

XO = 0 :
| XM = 0 :
| | XF = 0 :
| | | XB = 0 :
| | | | XG = 0 : 0
| | | | XG = 1 :
| | | | | XD = 0 :
| | | | | | XS = 0 : 0
| | | | | | XS = 1 :
| | | | | | | XC = 0 : 1
| | | | | | | XC = 1 :
| | | | | | | | XH = 0 : 0
| | | | | | | | XH = 1 : 1
| | | | | XD = 1 :
| | | | | | XE = 0 : 0
| | | | | | XE = 1 :
| | | | | | | XK = 0 : 0
| | | | | | | XK = 1 : 1
| | | XB = 1 :
| | | | XD = 0 : 0
| | | | XD = 1 :
| | | | | XI = 0 : 0
| | | | | XI = 1 :
| | | | | | XG = 0 : 1
| | | | | | XG = 1 : 0
| | XF = 1 : 0
| XM = 1 :
| | XB = 0 :
| | | XD = 0 :
| | | | XG = 0 :
| | | | | XF = 0 : 0
| | | | | XF = 1 :
| | | | | | XJ = 0 :
| | | | | | | XN = 0 : 1
| | | | | | | XN = 1 :
| | | | | | | | XE = 0 :
| | | | | | | | | XK = 0 : 0
| | | | | | | | | XK = 1 : 1
| | | | | | | | XE = 1 : 0
| | | | | | XJ = 1 :
| | | | | | | XC = 0 :
| | | | | | | | XT = 0 :
| | | | | | | | | XL = 0 :
| | | | | | | | | | XE = 0 :
| | | | | | | | | | | XI = 0 : 0
| | | | | | | | | | | XI = 1 : 1
| | | | | | | | | | 